In [28]:
#automatically reload stuff
%reload_ext autoreload
%autoreload 2
from SymptomPreprocessing import *
from SpatialPreprocessing import *
import Preprocessing as P
import Cluster
import Metrics
import re
import RPorts
import Utils
from warnings import simplefilter 
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.decomposition import PCA
from sklearn.calibration import CalibratedClassifierCV

from scipy.stats import chi2

import statsmodels.api as sm

simplefilter(action='ignore')
pd.set_option('display.max_rows', 1000)

In [2]:
rds = RadDataset()
rds.get_value_array(rds.dvh_df.columns,as_df=True)

spellchecking...
renaming things
filtering pateints
adding nan values for missing organs
adding histograms


,Esophagus_V30,Esophagus_V25,Esophagus_D25,Esophagus_D5,Esophagus_D97,Esophagus_D10,Esophagus_D15,Esophagus_V10,Esophagus_D30,Esophagus_D20,...,Glottic_Area_D90,Glottic_Area_D99,Glottic_Area_D80,Glottic_Area_volume,Glottic_Area_V50,Glottic_Area_V45,Glottic_Area_V55,Glottic_Area_V75,Glottic_Area_V20,Glottic_Area_D70
id,,,,,,,,,,,,,,,,,,,,,
2.0,78.937500,82.937500,44.437500,53.218750,4.710938,51.625000,49.781250,91.562500,42.531250,47.093750,...,16.500000,15.648438,17.343750,1058.807373,1.166992,5.058594,0.000000,0.0,61.093750,18.656250
6.0,70.312500,72.437500,41.437500,46.125000,2.189453,44.968750,44.031250,78.375000,40.375000,42.781250,...,29.343750,28.125000,30.515625,848.693848,0.000000,2.427734,0.000000,0.0,100.000000,31.093750
7.0,49.062500,58.031250,34.062500,38.687500,0.020004,37.625000,36.562500,70.312500,33.125000,35.250000,...,27.703125,27.343750,28.156250,902.252197,0.000000,0.000000,0.000000,0.0,100.000000,28.390625
8.0,29.531250,44.750000,30.609375,32.781250,4.468750,32.156250,31.703125,61.531250,29.937500,31.203125,...,7.859375,6.691406,9.328125,536.845779,0.000000,0.000000,0.000000,0.0,19.156250,9.812500
9.0,59.406250,73.375000,43.531250,54.468750,14.562500,51.437500,48.593750,100.000000,41.281250,46.031250,...,5.820312,5.621094,6.011719,1340.507126,0.000000,0.000000,0.000000,0.0,30.218750,6.218750
10.0,56.812500,60.843750,40.718750,45.718750,3.869141,44.218750,43.187500,75.187500,38.906250,42.000000,...,29.406250,25.453125,32.406250,1627.349854,31.390625,37.218750,21.015625,0.0,100.000000,35.718750
11.0,0.000000,20.859375,22.625000,26.812500,6.148438,26.156250,25.625000,42.812500,19.750000,25.093750,...,8.773438,7.589844,9.546875,692.138672,0.000000,0.000000,0.000000,0.0,7.738281,10.750000
12.0,0.000000,0.000000,2.160156,3.900391,0.010002,3.279297,2.839844,0.000000,1.940430,2.460938,...,1.629883,1.480469,1.769531,1650.810242,0.000000,0.000000,0.000000,0.0,12.937500,1.940430
13.0,74.437500,75.875000,41.031250,42.656250,0.020004,42.218750,41.812500,80.625000,40.656250,41.406250,...,7.031250,6.078125,7.898438,793.933868,0.000000,0.000000,0.000000,0.0,13.515625,8.757812


In [3]:
def rds_to_df(rds):
    sd_df = P.get_merged_symp_dose_df(rds,roll_up=True)
    sd_df = P.get_symptoms_max(sd_df)
    sd_df =  P.add_sd_dose_clusters(sd_df)
    sd_df = P.add_sd_symptom_clusters(sd_df)
    return sd_df
sddf = rds_to_df(rds)
sddf

before drop count 823
after drop count 535
error (%) 0.7424412531907885
[('severe_6wk_symptoms', 0.009345794392523364), ('severe_late_symptoms', 0.014953271028037384), ('moderate_6wk_symptoms', 0.052336448598130844), ('moderate_late_symptoms', 0.04672897196261682), ('mild_6wk_symptoms', 0.19626168224299065), ('mild_late_symptoms', 0.17009345794392525)]
dose ids: 484 syptom ids: 535 merged ids: 349
using default similarity


,D10,D15,D2,D20,D25,D30,D35,D40,D45,D5,...,symptoms_work_max_all,symptoms_work_max_treatment,symptoms_work_max_post,symptoms_work_max_late,symptoms_work_max_change_all,symptoms_work_max_change_treatment,symptoms_work_max_change_post,symptoms_work_max_change_late,dose_clusters,symptom_clusters
id,,,,,,,,,,,,,,,,,,,,,
7,"[37.625, 37.15625, 60.3125, 66.3125, 49.4375, ...","[36.5625, 36.75, 59.96875, 64.0625, 48.375, 49...","[39.96875, 38.0625, 60.90625, 71.8125, 52.1562...","[35.25, 36.34375, 59.71875, 62.71875, 47.34375...","[34.0625, 35.90625, 59.40625, 61.53125, 46.562...","[33.125, 35.5, 59.1875, 60.34375, 45.6875, 43....","[32.28125, 35.03125, 58.90625, 59.03125, 44.75...","[31.609375, 34.625, 58.625, 57.71875, 43.96875...","[30.84375, 34.1875, 58.3125, 56.40625, 43.1875...","[38.6875, 37.625, 60.625, 71.0625, 50.75, 53.8...",...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2,2
8,"[32.15625, 32.0625, 56.6875, 65.375, 22.5, 31....","[31.703125, 31.109375, 55.75, 63.34375, 21.312...","[33.4375, 34.09375, 58.28125, 68.75, 33.6875, ...","[31.203125, 30.25, 54.5625, 61.34375, 17.84375...","[30.609375, 29.40625, 52.375, 60.03125, 14.367...","[29.9375, 28.390625, 50.5625, 59.5, 13.140625,...","[28.96875, 27.453125, 49.46875, 58.96875, 11.7...","[27.1875, 26.46875, 48.59375, 58.53125, 11.203...","[24.875, 25.453125, 48.0, 58.125, 10.9609375, ...","[32.78125, 33.125, 57.65625, 67.375, 25.3125, ...",...,3.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,2,2
9,"[51.4375, 27.515625, 51.4375, 51.4375, 18.6875...","[48.59375, 26.671875, 48.59375, 48.59375, 15.0...","[56.65625, 29.5625, 56.65625, 56.65625, 27.312...","[46.03125, 25.9375, 46.03125, 46.03125, 13.0, ...","[43.53125, 25.296875, 43.53125, 43.53125, 11.3...","[41.28125, 24.671875, 41.28125, 41.28125, 10.3...","[39.15625, 24.046875, 39.15625, 39.15625, 9.82...","[37.0, 23.359375, 37.0, 37.0, 9.4609375, 11.28...","[35.125, 22.625, 35.125, 35.125, 9.109375, 9.3...","[54.46875, 28.625, 54.46875, 54.46875, 23.3125...",...,3.0,3.0,1.0,0.0,3.0,3.0,1.0,0.0,1,1
10,"[44.21875, 35.625, 64.75, 55.9375, 52.84375, 5...","[43.1875, 34.28125, 62.6875, 54.3125, 50.78125...","[47.84375, 39.21875, 68.5625, 58.15625, 57.406...","[42.0, 33.125, 59.53125, 52.84375, 48.875, 46....","[40.71875, 32.34375, 58.65625, 51.4375, 47.343...","[38.90625, 31.625, 58.3125, 50.0, 46.5, 44.687...","[36.875, 30.921875, 57.8125, 48.75, 46.125, 43...","[35.5625, 30.1875, 56.6875, 47.6875, 45.75, 43...","[33.96875, 29.4375, 55.03125, 46.4375, 45.3125...","[45.71875, 37.34375, 66.6875, 57.46875, 55.25,...",...,4.0,4.0,1.0,1.0,4.0,4.0,1.0,1.0,3,4
11,"[26.15625, 31.984375, 45.40625, 57.125, 29.265...","[25.625, 31.359375, 45.15625, 56.875, 26.48437...","[27.5625, 33.25, 45.84375, 57.65625, 36.75, 44...","[25.09375, 30.71875, 45.0, 56.65625, 22.828125...","[22.625, 30.15625, 44.8125, 56.46875, 19.85937...","[19.75, 29.65625, 44.625, 56.21875, 17.640625,...","[14.1484375, 29.234375, 44.46875, 55.9375, 15....","[11.09375, 28.84375, 44.21875, 55.6875, 14.210...","[9.1015625, 28.46875, 44.03125, 55.4375, 12.53...","[26.8125, 32.71875, 45.625, 57.40625, 33.03125...",...,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,2,1
13,"[42.21875, 29.625, 60.46875, 48.6875, 19.5, 41...","[41.8125, 29.046875, 58.34375, 48.15625, 14.52...","[43.0, 31.0, 70.625, 54.09375, 35.5625, 51.0, ...","[41.40625, 28.46875, 56.4375, 47.8125, 12.9921...","[41.03125, 27.859375, 54.5, 47.59375, 10.29687...","[40.65625, 27.234375, 52.78125, 47.375, 9.3593...","[40.3125, 26.359375, 50.9375, 47.1875, 8.67187...","[39.90625, 25.09375, 49.4375, 47.0, 8.3984375,...","[39.4375, 24.015625, 48.21875, 46.8125, 7.6484...","[42.65625, 30.375, 65.375, 52.125, 25.234375, ...",...,9.0,9.0,2.0,2.0,9.0,9.0,2.0,2.0,2,3
15,"[44.90625, 33.28125, 47.75, 57.71875, 43.625, ...","[44.0, 32.75, 47.5625, 57.21875, 40.40625, 48....","[46.5, 34.9375, 48.375, 58.71875, 46.875, 51.9...","[43.0625, 32.28125, 47.375, 56.625, 37.15625, ...","[42.03125, 31.8125, 47.25, 56.03125, 31.1875, ...","[41.03125, 31.046875, 47.125, 55.1875, 21.9

In [17]:
def add_organsubset_dose_clusters(df,
                                  features=['V35','V40','V45','V50','V55','V60','V65'],
                                  n_clusters =4,
                                 ):
    df = df.copy()
    for k,v in Const.symptom_organ_map.items():
        df = P.add_sd_dose_clusters(df,
                                       n_clusters=n_clusters,
                                       organ_subset=v,
                                       prefix=k+'_',
                                       features = features,
                                      )
    return df

sddf = add_organsubset_dose_clusters(sddf)

In [26]:
def add_dose_limits(df,organ_list=None):
    #dose limits as binary values from https://applications.emro.who.int/imemrf/Rep_Radiother_Oncol/Rep_Radiother_Oncol_2013_1_1_35_48.pdf
    #not inlcudeing other stuff like eyes at this time
    #also, my max dose is weird so I'm using V10 for that because I feel like that makes sense
    #using the 
    if organ_list is None:
        organ_list = Const.organ_list[:]
    df = df.copy()
    original_cols = set(df.columns)
    getval = lambda organ,param: df[param].apply(lambda x: x[organ_list.index(organ)])
    get_lr_val = lambda organ,param: np.maximum(getval('Lt_'+organ,param),getval('Rt_'+organ,param))
    
    maxdose_var = 'max_dose'
    #symptomatic necrosis
    df['Brainstem_limit'] = (getval('Brainstem','D2') > 59) | (getval('Brainstem','D99') > 54)
    
    #myelopathy
    df['Spinal_Cord_limit'] = getval('Spinal_Cord',maxdose_var) > 50
    
    #xerostomia. >25 for 1 or >20 for both
    df['Parotid_Gland_limit'] = (get_lr_val('Parotid_Gland','mean_dose') > 20) | (getval('Lt_Parotid_Gland','mean_dose') > 25) | (getval('Rt_Parotid_Gland','mean_dose') > 25)
    
    #there is 50 for PEG tube and 60 for aspiration so i'll do 50
    for o in ['IPC','MPC',"SPC"]:
        df[o+"_limit"] = getval(o,'mean_dose') > 50
        df[o+"_limit2"] = getval(o,'mean_dose') > 60
    
    #ORN?
    df['Mandible_limit'] = getval('Mandible',maxdose_var) > 70
    
    #edema
    df['Larynx_limit'] = getval('Larynx','V50') > 27
    
    #Nerve damage
    df['Brachial_Plexus_limit'] = get_lr_val('Brachial_Plexus',maxdose_var) > 60
    
    #Esophagitus
    elimits = [('V35',50),('V50',40),('V70',20),('V60',30)]
    df['Esophagus_limit'] = np.stack([(getval('Esophagus',v) > lim) for v,lim in elimits]).sum(axis=0) > 0
    
#     print(df[[c for c in df.columns if  c not in original_cols]].sum())
    return df

sddf = add_dose_limits(sddf)
sddf[[c for c in sddf.columns if 'limit' in c]].mean()

Brainstem_limit          0.002865
Spinal_Cord_limit        0.057307
Parotid_Gland_limit      0.851003
IPC_limit                0.180516
IPC_limit2               0.060172
MPC_limit                0.567335
MPC_limit2               0.306590
SPC_limit                0.704871
SPC_limit2               0.469914
Mandible_limit           0.558739
Larynx_limit             0.189112
Brachial_Plexus_limit    0.621777
Esophagus_limit          0.085960
dtype: float64

In [18]:
def var_test(df, testcol, ycol,xcols, model = None,regularize = False,scale=True):
    y = df[ycol]
    if testcol not in xcols:
        xcols = xcols + [testcol]
    x = df[xcols].astype(float)
    if regularize:
        for col in xcols:
            x[col] = (x[col] - x[col].mean())/(x[col].std()+ .01)
    if scale:
        for col in xcols:
            x[col] = (x[col] - x[col].min())/(x[col].max() - x[col].min())
    for col in xcols:
        if x[col].std() < .00001:
            x = x.drop(col,axis=1)
    x2 = x.copy()
    x2 = x2.drop(testcol,axis=1)
    if model is None:
        model = sm.Logit
        
    logit = model(y,x)
    logit_res = logit.fit(maxiter=500,disp=False,method='bfgs')
    
    logit2 = model(y,x2)
    logit2_res = logit2.fit(maxiter=500,disp=False,method='bfgs')
    
    llr_stat = 2*(logit_res.llf - logit2_res.llf)
    llr_p_val = chi2.sf(llr_stat,1)
    
    aic_diff = logit_res.aic - logit2_res.aic
    bic_diff = logit_res.bic - logit2_res.bic
    
    results = {
        'ttest_pval': logit_res.pvalues[testcol],
        'ttest_tval': logit_res.tvalues[testcol],
        'lrt_pval': llr_p_val,
        'aic_diff': aic_diff,
        'bic_diff': bic_diff
    }
    return results

temp = sddf.copy()
temp['total_dose'] = temp.mean_dose.apply(lambda x: np.sum(x))
clusters = [c for c in temp.columns if 'dose_cluster' in c]
covarslist = [
    ['Parotid_Gland_limit',
#           'IPC_limit','MPC_limit','SPC_limit',
          't4','n3','hpv','total_dose',
          "BOT","Tonsil",
         ],
   ['Parotid_Gland_limit',
          'IPC_limit','MPC_limit','SPC_limit',
          't4','n3','hpv','total_dose',
          "BOT","Tonsil",
         ] 
]
digest_cors = []
for covname, covars in zip(['clinical_subsites','clinical_limits_subsites'])
for clust in clusters:
    for clustval in temp[clust].unique():
        if clustval == 0:
            continue
        temp['x'] = temp[clust] == clustval
        temp_res = var_test(temp,'x','digest_increase',covars)
        temp_res['x'] = clust+'_'+str(clustval)
        temp_res['outcome'] = 'digest_increase'
        digest_cors.append(temp_res)
pd.DataFrame(digest_cors).sort_values('lrt_pval',ascending=True).to_csv(Const.data_dir+'cluster_digest_lrt_results.csv',index=False)
pd.read_csv(Const.data_dir+'cluster_digest_lrt_results.csv')

,ttest_pval,ttest_tval,lrt_pval,aic_diff,bic_diff,x,outcome
0,0.009211,2.604109,0.000833,-9.165819,-5.310747,soft_mouth_dose_clusters_3,digest_increase
1,0.009818,2.582164,0.003822,-6.366682,-2.511610,whole_mouth_dose_clusters_3,digest_increase
2,0.013992,2.457475,0.010513,-4.545782,-0.690710,upper_mouth_dose_clusters_3,digest_increase
3,0.929149,-0.088916,0.013232,-4.137789,-0.282717,shoulders_dose_clusters_1,digest_increase
4,0.018806,2.349346,0.023091,-3.161674,0.693398,chew_alt_dose_clusters_3,digest_increase
5,0.047819,-1.978976,0.024747,-3.041496,0.813575,upper_mouth_dose_clusters_1,digest_increase
6,0.039947,2.054294,0.031187,-2.642664,1.212408,lower_mouth_dose_clusters_3,digest_increase
7,0.923950,-0.095459,0.035980,-2.398053,1.457019,whole_mouth_dose_clusters_2,digest_increase
8,0.919579,-0.100964,0.056344,-1.641845,2.213227,lower_mouth_dose_clusters_2,digest_increase
9,0.053939,1.927330,0.057713,-1.601906,2.253166,dose_clusters_2,digest_increase


In [86]:
def multivar_tests(df,
                   cluster_names=None,
                   covariate_lists=None, 
                   outcomes = None, 
                   thresholds = [3,5,7]):
    #one-hot encode stuff
    for col in ['subsite']:
        if col in df.columns:
            df = Utils.onehotify(df,[c for c in df.columns if c != col])
    df['total_dose'] = df.mean_dose.apply(lambda x: np.sum(x))
    if cluster_names is None:
        cluster_names = [c for c in df.columns if 'dose_cluster' in c]
    if outcomes is None:
        outcomes = [c for c in df.columns if 'symptoms_' in c and ('_late' in c or '_post' in c)]
        soi = ['pain','teeth','drymouth','voice','mucus','vomit','choke','nausea','mucositis']
        outcomes = [c for c in outcomes if np.any([s in c for s in soi])]
    if covariate_lists is None:
#         limits = [c for c in df.columns if '_limit' in c and 'Brainstem' not in c]
        limits = ['Parotid_Gland_limit']
        pc_limits = ['IPC_limit','MPC_limit','SPC_limit']
        clinical = ['t4','n3','hpv','total_dose']
        subsites = ['BOT','Tonsil']
#         all_subsites = [c for c in df.columns if 'subsite_' in c]
        covariate_lists = {
            'clinical':clinical,
            'clinical_parotid': clinical+limits,
            'clinical_subsites': clinical+subsites,
            'clinical_parotid_subsites': limits+clinical+subsites,
            'clinical_limits': clinical+limits+pc_limits,
            'clinical_limits_subsites': clinical+limits+pc_limits+subsites,
    }
    result_list = []
    df = df.copy()
    for cluster_name in cluster_names:
        for cluster in df[cluster_name].unique():
            if cluster == 0:
                continue
            df['x'] = df[cluster_name] == cluster
            for outcome in outcomes:
                for threshold in thresholds:
                    df['y'] = df[outcome] >= threshold
                    for covname, covariates in covariate_lists.items():
                        try:
                            res = var_test(df,'x','y',covariates)
                            res['x'] = cluster_name+'_'+str(cluster)
                            res['y'] = outcome
                            if df.y.sum() < len(covariates)/10:
                                print('skipping',outcome,covname,df.y.sum())
                                continue
                            res['threshold']= threshold
                            res['covariates']=covname
                            if res['lrt_pval'] < .05 and covname == 'clinical_limits_subsites':
                                print('--------')
                                print(res['x'],res['y'],res['threshold'],res['lrt_pval'])
                                print('--------')
                            result_list.append(res)
                        except Exception as e:
                            print()
                            print(e)
                            print(cluster,outcome)
                            print()
    return result_list

soft_cluster_results = multivar_tests(sddf)
cluster_results  = pd.DataFrame(soft_cluster_results).sort_values('lrt_pval',ascending=True)
# cluster_results = pd.read_csv(Const.data_dir+'/cluster_lrt_results_03072022.csv')
cluster_results.sort_values('ttest_tval')

--------
dose_clusters_1 symptoms_mucositis_max_post 5 0.027039997084458495
--------
--------
dose_clusters_1 symptoms_mucositis_max_late 5 0.012389186586796067
--------
--------
dose_clusters_1 symptoms_mucositis_max_change_post 5 0.0113590191713361
--------
--------
dose_clusters_1 symptoms_mucositis_max_change_late 5 0.026122944950323922
--------
--------
dose_clusters_1 symptoms_pain_max_late 3 0.022444721216975893
--------
--------
dose_clusters_1 symptoms_pain_max_late 5 0.0028617903688485263
--------
--------
dose_clusters_1 symptoms_pain_max_change_late 3 0.026568929931848423
--------
--------
dose_clusters_1 symptoms_voice_max_late 3 0.03790326712617236
--------
--------
dose_clusters_1 symptoms_voice_max_change_post 3 0.048274821011482436
--------
--------
dose_clusters_1 symptoms_voice_max_change_late 3 0.027782990139435897
--------
--------
dose_clusters_3 symptoms_pain_max_late 3 0.03320107674244195
--------
--------
swallow_dose_clusters_2 symptoms_choke_max_late 3 0.0324

--------
upper_mouth_dose_clusters_1 symptoms_mucus_max_change_post 3 0.012495166474890575
--------
--------
upper_mouth_dose_clusters_1 symptoms_nausea_max_post 3 0.0003766465950458629
--------
--------
upper_mouth_dose_clusters_1 symptoms_nausea_max_post 5 0.0006311643226854192
--------
--------
upper_mouth_dose_clusters_1 symptoms_nausea_max_post 7 0.01141455205623044
--------
--------
upper_mouth_dose_clusters_1 symptoms_nausea_max_change_post 3 0.00010532071939774193
--------
--------
upper_mouth_dose_clusters_1 symptoms_nausea_max_change_post 5 0.000572971264489915
--------
--------
upper_mouth_dose_clusters_1 symptoms_nausea_max_change_post 7 0.0021203975263383475
--------
--------
upper_mouth_dose_clusters_1 symptoms_nausea_max_change_late 3 0.04671346967377428
--------
--------
upper_mouth_dose_clusters_1 symptoms_pain_max_post 5 0.007238846919734492
--------
--------
upper_mouth_dose_clusters_1 symptoms_pain_max_post 7 0.0012396595436114512
--------
--------
upper_mouth_dose_

--------
lower_mouth_dose_clusters_1 symptoms_pain_max_late 5 0.032337865909442605
--------
--------
lower_mouth_dose_clusters_1 symptoms_pain_max_late 7 0.026100432915423376
--------
--------
lower_mouth_dose_clusters_1 symptoms_voice_max_post 3 0.01186309887108378
--------
--------
lower_mouth_dose_clusters_1 symptoms_voice_max_late 3 0.008166948018056
--------
--------
lower_mouth_dose_clusters_1 symptoms_voice_max_change_post 3 0.007103314023955218
--------
--------
lower_mouth_dose_clusters_1 symptoms_voice_max_change_late 3 0.0026001640735067236
--------
--------
lower_mouth_dose_clusters_1 symptoms_voice_max_change_late 7 0.03823176812675198
--------
--------
lower_mouth_dose_clusters_2 symptoms_drymouth_max_late 5 0.04812098096859457
--------
--------
lower_mouth_dose_clusters_2 symptoms_pain_max_late 3 0.023225888387514222
--------
--------
lower_mouth_dose_clusters_2 symptoms_pain_max_late 5 0.02074904247372266
--------
--------
lower_mouth_dose_clusters_2 symptoms_pain_max_l

--------
shoulders_dose_clusters_3 symptoms_mucositis_max_change_late 5 0.023992671006184105
--------
--------
shoulders_dose_clusters_3 symptoms_nausea_max_late 7 0.03204108787506237
--------
--------
shoulders_dose_clusters_3 symptoms_nausea_max_change_late 5 0.039803250426272786
--------
--------
shoulders_dose_clusters_3 symptoms_nausea_max_change_late 7 0.012887769894201444
--------
--------
shoulders_dose_clusters_3 symptoms_pain_max_late 5 0.04195554161923483
--------
--------
shoulders_dose_clusters_3 symptoms_pain_max_change_post 5 0.014788991029307594
--------
--------
shoulders_dose_clusters_3 symptoms_voice_max_change_post 7 0.0035911397263034754
--------
--------
shoulders_dose_clusters_3 symptoms_voice_max_change_late 7 0.001099954005279164
--------
--------
shoulders_dose_clusters_2 symptoms_mucositis_max_change_post 5 0.01969290767229595
--------
--------
shoulders_dose_clusters_2 symptoms_mucositis_max_change_post 7 0.021912380137577646
--------
--------
shoulders_dose

--------
soft_mouth_dose_clusters_1 symptoms_teeth_max_late 7 0.04740795525179758
--------
--------
soft_mouth_dose_clusters_1 symptoms_vomit_max_late 5 0.014817413600100054
--------
--------
soft_mouth_dose_clusters_1 symptoms_vomit_max_late 7 0.014817413600100054
--------
--------
soft_mouth_dose_clusters_1 symptoms_vomit_max_change_late 5 0.014817413600100054
--------
--------
soft_mouth_dose_clusters_1 symptoms_vomit_max_change_late 7 0.014817413600100054
--------
--------
PCs_dose_clusters_3 symptoms_mucus_max_late 7 0.03486774889598074
--------
--------
PCs_dose_clusters_3 symptoms_mucus_max_change_late 3 0.03491598867273265
--------
--------
PCs_dose_clusters_3 symptoms_mucus_max_change_late 5 0.04911880089603849
--------
--------
PCs_dose_clusters_3 symptoms_mucus_max_change_late 7 0.047676293046208025
--------
--------
PCs_dose_clusters_3 symptoms_teeth_max_change_post 3 0.02832984837467969
--------
--------
PCs_dose_clusters_3 symptoms_teeth_max_change_post 5 0.00120695777423

--------
chew_alt_dose_clusters_2 symptoms_drymouth_max_change_post 7 0.039063908128234584
--------
--------
chew_alt_dose_clusters_2 symptoms_mucositis_max_post 3 0.0329424719766918
--------
--------
chew_alt_dose_clusters_2 symptoms_mucositis_max_post 5 0.00730897220711075
--------
--------
chew_alt_dose_clusters_2 symptoms_mucositis_max_late 5 0.023952028266808163
--------
--------
chew_alt_dose_clusters_2 symptoms_mucositis_max_change_post 5 0.004191561148392126
--------
--------
chew_alt_dose_clusters_2 symptoms_mucositis_max_change_late 5 0.03496599900620178
--------
--------
chew_alt_dose_clusters_2 symptoms_pain_max_post 3 0.028688205913288627
--------
--------
chew_alt_dose_clusters_2 symptoms_pain_max_post 7 0.04096883341516331
--------
--------
chew_alt_dose_clusters_2 symptoms_voice_max_change_post 7 0.02395236636691117
--------
--------
chew_alt_dose_clusters_2 symptoms_vomit_max_post 3 0.042688637957871214
--------
--------
chew_alt_dose_clusters_2 symptoms_vomit_max_post

,ttest_pval,ttest_tval,lrt_pval,aic_diff,bic_diff,x,y,threshold,covariates
5081,0.000023,-4.234675,0.000007,-18.265487,-14.410415,upper_mouth_dose_clusters_1,symptoms_voice_max_change_post,3,clinical_limits_subsites
5079,0.000031,-4.163078,0.000010,-17.486780,-13.631708,upper_mouth_dose_clusters_1,symptoms_voice_max_change_post,3,clinical_parotid_subsites
5078,0.000055,-4.034294,0.000023,-15.961901,-12.106829,upper_mouth_dose_clusters_1,symptoms_voice_max_change_post,3,clinical_subsites
4861,0.000102,-3.885207,0.000012,-17.096085,-13.241013,upper_mouth_dose_clusters_1,symptoms_nausea_max_change_post,3,clinical_parotid
4864,0.000150,-3.790758,0.000025,-15.795519,-11.940447,upper_mouth_dose_clusters_1,symptoms_nausea_max_change_post,3,clinical_limits
...,...,...,...,...,...,...,...,...,...
14867,NaN,NaN,0.926140,1.991406,5.846478,whole_mouth_dose_clusters_1,symptoms_vomit_max_late,7,clinical_limits_subsites
8457,NaN,NaN,0.927222,1.991657,5.846729,chew_dose_clusters_2,symptoms_choke_max_late,7,clinical_parotid_subsites
11439,NaN,NaN,0.949060,1.995918,5.850990,throat_dose_clusters_1,symptoms_pain_max_change_late,5,clinical_parotid_subsites
21159,NaN,NaN,0.949060,1.995918,5.850990,throat_alt_dose_clusters_1,symptoms_pain_max_change_late,5,clinical_parotid_subsites


In [77]:
temp = cluster_results[(cluster_results.x == 'soft_mouth_dose_clusters_3')]
templist = []
for threshold in [5,7]:
    temp2 = temp[temp.threshold ==threshold]
    for outcome in ['post','change_post','late','change_late']:
        if threshold == 7 and 'change' in outcome:
            continue
        ocome = 'symptoms_drymouth_max_' + outcome
        temp3 = temp2[temp.y == ocome]
        for covname in ['clinical_parotid_subsites']:
            temp4 = temp3[temp.covariates == covname]
            templist.append(temp4[['y','threshold','covariates','lrt_pval','ttest_tval','aic_diff']])
pd.concat(templist,axis=0)

,y,threshold,covariates,lrt_pval,ttest_tval,aic_diff
7815,symptoms_drymouth_max_post,5,clinical_parotid_subsites,0.045385,1.986753,-2.004284
7833,symptoms_drymouth_max_change_post,5,clinical_parotid_subsites,0.013502,2.453701,-4.102055
7824,symptoms_drymouth_max_late,5,clinical_parotid_subsites,0.021263,2.288765,-3.305124
7842,symptoms_drymouth_max_change_late,5,clinical_parotid_subsites,0.005014,2.786961,-5.874234
7818,symptoms_drymouth_max_post,7,clinical_parotid_subsites,0.020656,2.304264,-3.355617
7827,symptoms_drymouth_max_late,7,clinical_parotid_subsites,0.014043,2.441463,-4.032761


In [85]:
def print_good_results(df,cluster = None,symptom='drymouth',covariates=None,threshold=None):
    test = df.sort_values('ttest_tval',ascending=False)
    test = test[(test.lrt_pval < .1) & (test.ttest_tval > 0)].drop(['ttest_pval','bic_diff'],axis=1)
    if cluster is not None:
        test = test[test.x.apply(lambda x: cluster in x)]
    if symptom is not None:
        test = test[test.y.apply(lambda x: symptom in x)]
    if covariates is not None:
        test = test[test.covariates == covariates]
    if threshold is not None:
        test =test[test.threshold == threshold]
    return test
print_good_results(cluster_results,
                   cluster='soft_mouth',
                   symptom=None,
                   covariates='clinical_limits_subsites',
                  )

,ttest_tval,lrt_pval,aic_diff,x,y,threshold,covariates
8294,3.133536,0.001473,-8.111964,soft_mouth_dose_clusters_2,symptoms_pain_max_late,5,clinical_limits_subsites
7844,2.496073,0.011926,-4.321889,soft_mouth_dose_clusters_3,symptoms_drymouth_max_change_late,5,clinical_limits_subsites
8285,2.485630,0.013154,-4.148340,soft_mouth_dose_clusters_2,symptoms_pain_max_post,5,clinical_limits_subsites
8282,2.456203,0.014100,-4.025634,soft_mouth_dose_clusters_2,symptoms_pain_max_post,3,clinical_limits_subsites
8291,2.365735,0.018421,-3.555623,soft_mouth_dose_clusters_2,symptoms_pain_max_late,3,clinical_limits_subsites
8024,2.213563,0.016427,-3.756620,soft_mouth_dose_clusters_3,symptoms_teeth_max_change_late,5,clinical_limits_subsites
7826,2.211384,0.026047,-2.952891,soft_mouth_dose_clusters_3,symptoms_drymouth_max_late,5,clinical_limits_subsites
7841,2.185585,0.026845,-2.900742,soft_mouth_dose_clusters_3,symptoms_drymouth_max_change_late,3,clinical_limits_subsites
8300,2.179664,0.029961,-2.711546,soft_mouth_dose_clusters_2,symptoms_pain_max_change_post,3,clinical_limits_subsites
7835,2.164198,0.029544,-2.735635,soft_mouth_dose_clusters_3,symptoms_drymouth_max_change_post,5,clinical_limits_subsites


In [52]:
print_good_results(cluster_results,
                   covariates='clinical_limits',
                  )

,ttest_tval,lrt_pval,aic_diff,x,y,threshold,covariates
2644,2.004737,0.045541,-1.998479,upper_mouth_dose_clusters_3,symptoms_drymouth_max_post,7,clinical_limits
5236,2.089815,0.036360,-2.380118,throat_dose_clusters_3,symptoms_drymouth_max_post,7,clinical_limits
2980,2.116994,0.033265,-2.532081,lower_mouth_dose_clusters_3,symptoms_drymouth_max_change_post,5,clinical_limits
7822,2.211127,0.025728,-2.974218,soft_mouth_dose_clusters_3,symptoms_drymouth_max_post,5,clinical_limits
7828,2.251668,0.023449,-3.134932,soft_mouth_dose_clusters_3,symptoms_drymouth_max_post,7,clinical_limits
2308,2.630666,0.006806,-5.323536,upper_mouth_dose_clusters_1,symptoms_drymouth_max_post,3,clinical_limits
1030,2.692760,0.005538,-5.694803,swallow_dose_clusters_2,symptoms_drymouth_max_change_post,3,clinical_limits
7840,2.771039,0.005093,-5.846178,soft_mouth_dose_clusters_3,symptoms_drymouth_max_change_post,5,clinical_limits


In [157]:
# pd.DataFrame(soft_cluster_results).to_csv(Const.data_dir+'/cluster_lrt_results_03072022.csv',index=False)

In [33]:
def select_single_organ_cluster_effects(df,
                                        outcome,
                                        base_organs=None,
                                        covars=None,
                                        organ_list=None):
    if base_organs is None:
        base_organs = []
    if organ_list is None:
        organ_list = Const.organ_list
    olists = [base_organs] if len(base_organs) > 0 else []
    for o in organ_list:
        if o in base_organs:
            continue
        if 'Rt_' in o:
            continue
        new_list = [o]
        if len(base_organs) > 0:
            new_list = new_list + base_organs
        if 'Lt_' in o:
            new_list.append(o.replace('Lt_','Rt_'))
        olists.append(new_list)
    if covars is None:
        covars = [
            'Parotid_Gland_limit',
          'IPC_limit','MPC_limit','SPC_limit',
          't4','n3','hpv','total_dose',
          "BOT","Tonsil",
         ]
    df = df.copy()
    df['total_dose'] = df.mean_dose.apply(lambda x: np.sum(x))
    results = []
    base_pval = 1
    for olist in olists:
        prefix = '_'.join(olist)+'_'
        nclust = 4
        df  = P.add_sd_dose_clusters(df,
                                     features = ['V35','V40','V45','V50','V55','V60','V65'],
                                     organ_subset=olist,
                                     prefix=prefix,
                                     n_clusters=nclust
            )
        clustname = prefix+'dose_clusters'
        for cval in df[clustname].unique():
            if cval == 0:
                continue
            df['x'] = df[clustname] == cval
            entry = {
                'x': clustname,
                'outcome':outcome,
                'olist':olist,
            }
            res = var_test(df,'x',outcome,covars)
            if ''.join(olist) == ''.join(base_organs):
                base_pval = res['lrt_pval']
            entry['pval_change'] = base_pval - res['lrt_pval']
            for k,v in res.items():
                entry[k] = v
            results.append(entry)
    results=pd.DataFrame(results).sort_values('pval_change',ascending=False)
    return results

def run_forward_search(df,
                       outcome,
                       outcome_threshold,
                       organ_list=None,
                       max_depth = 4,
                      ):
    if organ_list is None:
        organ_list = [
        'Esophagus',#
         'Cricopharyngeal_Muscle',#
         'Cricoid_cartilage',#
         'IPC',#
         'MPC',#
         'Larynx',#
         'Thyroid_cartilage',#
         'Lt_Sternocleidomastoid_M',
         'Lt_Mastoid',
         'Lt_Parotid_Gland',
         'Lt_Submandibular_Gland',
         'Lt_Medial_Pterygoid_M',
         'Lt_Lateral_Pterygoid_M',
         'Lt_Masseter_M',
         'Supraglottic_Larynx',#
         'SPC',#
         'Soft_Palate',#
         'Genioglossus_M',#
         'Tongue',#
         'Lt_Ant_Digastric_M',
         'Mylogeniohyoid_M',#
         'Hard_Palate',
         'Glottic_Area',
                 ]
    temp = df.copy()
    temp['outcome'] = temp[outcome] >= outcome_threshold
    filter_pval = lambda dframe: dframe[(dframe.pval_change > 0) & (dframe.lrt_pval < .01)]
    temp_res = select_single_organ_cluster_effects(temp,'outcome',organ_list=organ_list)
    top_values = filter_pval(temp_res)
    frontier = top_values.olist.values.tolist()
    dflist = [top_values]
    curr_df = pd.concat(dflist,axis=0,ignore_index=True)
    curr_best = 1
    while len(frontier) > 0:
        polist = frontier.pop()
        if len(polist) > max_depth:
            continue
        try:
            temp_res = select_single_organ_cluster_effects(temp,'outcome',
                                                           organ_list=organ_list,
                                                           base_organs=polist)
            top_values = filter_pval(temp_res)
            if len(top_values) >= 1:
                dflist.append(top_values)
                frontier.extend(top_values.olist.values.tolist())

            curr_df = pd.concat(dflist,axis=0,ignore_index=True)
            
            best = curr_df[['olist','lrt_pval']].sort_values('lrt_pval',ascending=True).iloc[0]
            if best.lrt_pval < curr_best:
                curr_best = best.lrt_pval
                print(best)
            else:
                print(len(frontier),'left',end='\r')
        except Exception as e:
            print('error in getting lrt results', e)
    return curr_df.sort_values('lrt_pval')

search_res_list= []
outcomes = [c for c in sddf.columns if 'symptoms_' in c and '_late' in c]
soi = ['pain','teeth','drymouth','voice','mucus','vomit','choke','nausea','mucositis']
outcomes = [c for c in outcomes if np.any([s in c for s in soi])]
for outcome in outcomes:
    for threshold in [3,5,7]:
        if (sddf[outcome] > threshold).mean() > .8 or (sddf[outcome] > threshold).mean() < .05:
            print('skipping',outcome,'=',threshold,(sddf[outcome] > threshold).mean())
            continue
        fr = run_forward_search(sddf,outcome,threshold)
        fr = fr[fr.lrt_pval <=  2*fr.lrt_pval.min()]
        fr['outcome'] = outcome
        print('round done')
        print(fr)
        print()
        search_res_list.append(fr)
pd.concat(search_res_list,axis=0,ignore_index=True)

olist       [SPC, Hard_Palate]
lrt_pval              0.003546
Name: 2, dtype: object
olist       [Cricopharyngeal_Muscle, Esophagus, Lt_Massete...
lrt_pval                                             0.000881
Name: 6, dtype: object
olist       [Lt_Submandibular_Gland, Lt_Sternocleidomastoi...
lrt_pval                                             0.000617
Name: 13, dtype: object
olist       [Larynx, Soft_Palate, SPC, Hard_Palate]
lrt_pval                                   0.000071
Name: 21, dtype: object
olist       [Lt_Ant_Digastric_M, Larynx, Soft_Palate, SPC,...
lrt_pval                                             0.000042
Name: 32, dtype: object
round done
                                                    x  \
32  Lt_Ant_Digastric_M_Larynx_Soft_Palate_SPC_Hard...   
79  Lt_Ant_Digastric_M_Larynx_Soft_Palate_SPC_Hard...   
21   Larynx_Soft_Palate_SPC_Hard_Palate_dose_clusters   
59  Lt_Ant_Digastric_M_IPC_Soft_Palate_SPC_Hard_Pa...   
44  Lt_Ant_Digastric_M_Glottic_Area_Soft_Palate_

round done
Empty DataFrame
Columns: [x, outcome, olist, pval_change, ttest_pval, ttest_tval, lrt_pval, aic_diff, bic_diff]
Index: []

round done
Empty DataFrame
Columns: [x, outcome, olist, pval_change, ttest_pval, ttest_tval, lrt_pval, aic_diff, bic_diff]
Index: []

olist       [Lt_Sternocleidomastoid_M, Rt_Sternocleidomast...
lrt_pval                                             0.002965
Name: 0, dtype: object
olist       [IPC, Tongue]
lrt_pval         0.001748
Name: 3, dtype: object
olist       [Lt_Mastoid, SPC, Hard_Palate, Tongue, Rt_Mast...
lrt_pval                                             0.000142
Name: 6, dtype: object
olist       [IPC, Esophagus, SPC, Hard_Palate, Tongue]
lrt_pval                                      0.000033
Name: 17, dtype: object
olist       [Esophagus, Lt_Medial_Pterygoid_M, IPC, Tongue...
lrt_pval                                             0.000023
Name: 58, dtype: object
round done
                                                    x  \
58  Esophagus

round done
                                                      x  \
3175  Lt_Mastoid_Genioglossus_M_Tongue_Soft_Palate_S...   
1606  Lt_Mastoid_Genioglossus_M_Tongue_Soft_Palate_S...   
538   Lt_Mastoid_Genioglossus_M_Soft_Palate_Tongue_S...   
1011  Lt_Mastoid_Genioglossus_M_Tongue_Soft_Palate_S...   
293   Lt_Mastoid_Genioglossus_M_Soft_Palate_Tongue_S...   
3510  Lt_Mastoid_Tongue_Genioglossus_M_Soft_Palate_S...   
2580  Lt_Mastoid_Genioglossus_M_Tongue_Soft_Palate_S...   
1941  Lt_Mastoid_Tongue_Genioglossus_M_Soft_Palate_S...   
2175  Lt_Mastoid_Thyroid_cartilage_MPC_Soft_Palate_S...   
3744  Lt_Mastoid_Thyroid_cartilage_MPC_Soft_Palate_S...   
934   Lt_Mastoid_Tongue_Soft_Palate_SPC_Rt_Mastoid_d...   
3098  Lt_Mastoid_Tongue_Soft_Palate_SPC_Rt_Mastoid_d...   
1529  Lt_Mastoid_Tongue_Soft_Palate_SPC_Rt_Mastoid_d...   
216   Lt_Mastoid_Soft_Palate_Tongue_SPC_Rt_Mastoid_d...   
2503  Lt_Mastoid_Tongue_Soft_Palate_SPC_Rt_Mastoid_d...   
461   Lt_Mastoid_Soft_Palate_Tongue_SPC_Rt_Ma

olist       [Supraglottic_Larynx]
lrt_pval                 0.001212
Name: 0, dtype: object
olist       [Mylogeniohyoid_M, Supraglottic_Larynx, Tongue]
lrt_pval                                           0.000675
Name: 19, dtype: object
round done
                                                    x  \
19  Mylogeniohyoid_M_Supraglottic_Larynx_Tongue_do...   
90  Mylogeniohyoid_M_Tongue_Supraglottic_Larynx_do...   
52  Mylogeniohyoid_M_Soft_Palate_Tongue_dose_clusters   
70  Mylogeniohyoid_M_Soft_Palate_Tongue_dose_clusters   
58  MPC_Cricopharyngeal_Muscle_Soft_Palate_Tongue_...   
76  MPC_Cricopharyngeal_Muscle_Soft_Palate_Tongue_...   
0                   Supraglottic_Larynx_dose_clusters   
20  Soft_Palate_Supraglottic_Larynx_Tongue_dose_cl...   
53  Supraglottic_Larynx_Soft_Palate_Tongue_dose_cl...   
91  Soft_Palate_Tongue_Supraglottic_Larynx_dose_cl...   
71  Supraglottic_Larynx_Soft_Palate_Tongue_dose_cl...   

                               outcome  \
19  symptoms_mucositis_max_

round done
                                                   x                  outcome  \
0  Lt_Sternocleidomastoid_M_Rt_Sternocleidomastoi...  symptoms_mucus_max_late   
1                                  SPC_dose_clusters  symptoms_mucus_max_late   
3                           MPC_Tongue_dose_clusters  symptoms_mucus_max_late   
2                               Tongue_dose_clusters  symptoms_mucus_max_late   
4                Cricoid_cartilage_SPC_dose_clusters  symptoms_mucus_max_late   

                                               olist  pval_change  ttest_pval  \
0  [Lt_Sternocleidomastoid_M, Rt_Sternocleidomast...     0.993321    0.990711   
1                                              [SPC]     0.991318    0.013185   
3                                      [MPC, Tongue]     0.707514    0.017586   
2                                           [Tongue]     0.990539    0.011175   
4                           [Cricoid_cartilage, SPC]     0.252563    0.013378   

   ttest_tval  

olist       [Esophagus, Hard_Palate]
lrt_pval                    0.000734
Name: 3, dtype: object
olist       [Lt_Masseter_M, Esophagus, Hard_Palate, Rt_Mas...
lrt_pval                                             0.000299
Name: 4, dtype: object
round done
                                                     x  \
180  Hard_Palate_Lt_Masseter_M_Esophagus_Rt_Massete...   
4    Lt_Masseter_M_Esophagus_Hard_Palate_Rt_Massete...   
229  Hard_Palate_Lt_Masseter_M_Esophagus_Rt_Massete...   
247  Lt_Masseter_M_Hard_Palate_Esophagus_Rt_Massete...   
5     Glottic_Area_Esophagus_Hard_Palate_dose_clusters   
248   Glottic_Area_Hard_Palate_Esophagus_dose_clusters   

                           outcome  \
180  symptoms_pain_max_change_late   
4    symptoms_pain_max_change_late   
229  symptoms_pain_max_change_late   
247  symptoms_pain_max_change_late   
5    symptoms_pain_max_change_late   
248  symptoms_pain_max_change_late   

                                                 olist  pval_change  \


,x,outcome,olist,pval_change,ttest_pval,ttest_tval,lrt_pval,aic_diff,bic_diff
0,Lt_Ant_Digastric_M_Larynx_Soft_Palate_SPC_Hard...,symptoms_choke_max_late,"[Lt_Ant_Digastric_M, Larynx, Soft_Palate, SPC,...",0.510956,1.131872e-04,-3.860431,0.000042,-14.769537,-10.914465
1,Lt_Ant_Digastric_M_Larynx_Soft_Palate_SPC_Hard...,symptoms_choke_max_late,"[Lt_Ant_Digastric_M, Larynx, Soft_Palate, SPC,...",0.510956,1.131872e-04,-3.860431,0.000042,-14.769537,-10.914465
2,Larynx_Soft_Palate_SPC_Hard_Palate_dose_clusters,symptoms_choke_max_late,"[Larynx, Soft_Palate, SPC, Hard_Palate]",0.482429,1.246781e-04,-3.836740,0.000071,-13.780111,-9.925039
3,Lt_Ant_Digastric_M_IPC_Soft_Palate_SPC_Hard_Pa...,symptoms_choke_max_late,"[Lt_Ant_Digastric_M, IPC, Soft_Palate, SPC, Ha...",0.394700,1.530956e-04,-3.785994,0.000074,-13.707504,-9.852432
4,Lt_Ant_Digastric_M_Glottic_Area_Soft_Palate_SP...,symptoms_choke_max_late,"[Lt_Ant_Digastric_M, Glottic_Area, Soft_Palate...",0.374525,1.659219e-04,-3.765943,0.000082,-13.516458,-9.661386
5,Supraglottic_Larynx_Larynx_dose_clusters,symptoms_choke_max_late,"[Supraglottic_Larynx, Larynx]",0.008201,6.216659e-04,3.421981,0.000281,-11.193978,-7.338906
6,Supraglottic_Larynx_Larynx_dose_clusters,symptoms_choke_max_late,"[Supraglottic_Larynx, Larynx]",0.008201,6.216659e-04,3.421981,0.000281,-11.193978,-7.338906
7,Larynx_dose_clusters,symptoms_choke_max_late,[Larynx],0.999587,9.996936e-01,-0.000384,0.000413,-10.470258,-6.615186
8,Lt_Ant_Digastric_M_Cricoid_cartilage_Thyroid_c...,symptoms_choke_max_late,"[Lt_Ant_Digastric_M, Cricoid_cartilage, Thyroi...",0.025253,8.840400e-01,-0.145850,0.000442,-10.346845,-6.491773
9,Lt_Ant_Digastric_M_Cricoid_cartilage_Thyroid_c...,symptoms_choke_max_late,"[Lt_Ant_Digastric_M, Cricoid_cartilage, Thyroi...",0.025253,8.840400e-01,-0.145850,0.000442,-10.346845,-6.491773


In [57]:
pd.concat(search_res_list,axis=0,ignore_index=True).sort_values('lrt_pval').to_csv(Const.data_dir+'forward_search_results.csv',index=False)
odf = pd.read_csv(Const.data_dir+'forward_search_results.csv')
odf[odf.outcome.apply(lambda x: 'drymouth' in x)][['olist','lrt_pval']].values

array([["['Lt_Masseter_M', 'IPC', 'Esophagus', 'Lt_Mastoid', 'Rt_Mastoid', 'Rt_Masseter_M']",
        1.1265390201411528e-05],
       ["['Lt_Masseter_M', 'Esophagus', 'IPC', 'Lt_Mastoid', 'Rt_Mastoid', 'Rt_Masseter_M']",
        1.1265390201411528e-05],
       ["['Lt_Masseter_M', 'Esophagus', 'IPC', 'Lt_Mastoid', 'Rt_Mastoid', 'Rt_Masseter_M']",
        1.1265390201411528e-05],
       ["['Lt_Masseter_M', 'IPC', 'Esophagus', 'Lt_Mastoid', 'Rt_Mastoid', 'Rt_Masseter_M']",
        1.1265390201411528e-05],
       ["['Lt_Masseter_M', 'IPC', 'Esophagus', 'Lt_Mastoid', 'Rt_Mastoid', 'Rt_Masseter_M']",
        1.1265390201411528e-05],
       ["['Lt_Masseter_M', 'IPC', 'Esophagus', 'Lt_Mastoid', 'Rt_Mastoid', 'Rt_Masseter_M']",
        1.1265390201411528e-05],
       ["['Lt_Masseter_M', 'Esophagus', 'IPC', 'Lt_Mastoid', 'Rt_Mastoid', 'Rt_Masseter_M']",
        1.1265390201411528e-05],
       ["['Lt_Masseter_M', 'Esophagus', 'IPC', 'Lt_Mastoid', 'Rt_Mastoid', 'Rt_Masseter_M']",
        1.1265390